In [1]:
# 50,000개의 영화 평론 데이터를 불러옴.
# https://ai.stanford.edu/~amaas/data/sentiment/에서 Internet movie database를 받은 후
# 아래의 프로그램을 수행함.
import tarfile
tar = tarfile.open('C:/Users/ysp/Desktop/Deep Learning/aclImdb_v1.tar.gz') # gz파일이 저장된 경로로 변경
tar.extractall()

In [2]:
#!pip install PyPrind
# 자료를 행렬 형태로 바꿈.
import pyprind
import pandas as pd
import os
basepath = 'C://Users//ysp//Desktop//Deep Learning//aclImdb_v1//aclImdb'
labels = {'pos':1, 'neg':0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']    

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


In [3]:
# 자료의 순서를 임의로 뒤섞어 csv 파일로 저장. 
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('C://Users//ysp//Desktop//Deep Learning//movie_data.csv', index = False, encoding='utf-8')

In [4]:
# 저장된 파일 불러와 확인함.
import pyprind
import pandas as pd
import os
df = pd.read_csv('C://Users//ysp//Desktop//Deep Learning//movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [5]:
# 정보를 가지지 않은 것으로 판단 되는 것을 사전에 정리
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # text에서 <[^>]*>과 일치하는 데이터를 공백으로 바꾸는 명령어
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
    text)
    text = (re.sub('[\W]+', ' ', text.lower()) # 단어가 아닌 모든 기호는 공백으로 대체
    +' '.join(emoticons).replace('-', '')) # emoticons을 빈공간 뒤에 배치
    return text
df['review'] = df['review'].apply(preprocessor)
df['review']

0        in 1974 the teenager martha moxley maggie grac...
1        ok so i really like kris kristofferson and his...
2         spoiler do not read this if you think about w...
3        hi for all the people who have seen this wonde...
4        i recently bought the dvd forgetting just how ...
                               ...                        
49995    ok lets start with the best the building altho...
49996    the british heritage film industry is out of c...
49997    i don t even know where to begin on this one i...
49998    richard tyler is a little boy who is scared of...
49999    i waited long to watch this movie also because...
Name: review, Length: 50000, dtype: object

In [6]:
# 단어를 분류
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [7]:
# 단어의 원뿌리로 재표현
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [8]:
# 정보가 없는 단어 제거
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs alot')[-10:] if w not in stop]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ysp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'alot']

In [9]:
# 코드 출력시 불필요한 warning 메시지가 출력되지 않도록 함
# 출력하지 않을 warining 메시지의 종류를 지정할 수 있음 ('DeprecationWarning' 등)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [10]:
# 영화에 대한 평가를 positive or negative로 분류
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values 
y_test = df.loc[25000:, 'sentiment'].values

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
param_grid = [{'vect__ngram_range': [(1,1)],'vect__stop_words': ['english', None],'vect__tokenizer': [None,tokenizer_porter],
               'clf__penalty':['l2', 'none'],'clf__C': [1.0, 10.0, 100.0]}] 
lr_tfidf = Pipeline([('vect', tfidf),('clf', LogisticRegression(random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,scoring='accuracy',cv=5, verbose=1,n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train) 

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 70.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        no

In [11]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': None} 


In [12]:
# 검증데이터의 정밀도
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

CV Accuracy: 0.895


In [15]:
# 초모수 선택 후 시험데이터에 적용한 결과
print('Test Accuracy: %.3f' %  gs_lr_tfidf.score(X_test, y_test))

Test Accuracy: 0.897
